In [1]:
root_path = "/tf/home/sergio/Tesis"

In [2]:
import os
os.getcwd()

'/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [3]:
import sys
sys.path.append(root_path+"/TinyYOLOv3-Pedestrian-Detection")
from YOLOblocks import TinyYOLOv3,BasicBlock,PredictionLayer#,YOLOLossBasicBlock

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf

from skimage.io import imread,imshow
from skimage.transform import resize
import time
import os

from pycocotools.coco import COCO

In [4]:
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]
model = TinyYOLOv3(1,anchor_boxes=anchors,train=False,mode = "finetuning",obj_threshold=0.4)
model.build(batch_input_shape=(None,416,416,3))    
model.load_weights(root_path+'/weights_saved/pesos_finetuning_5521_30_50_epoch_nadam_0dot00001_msle_exp_mse_3anchors')

In [5]:
file = tf.keras.utils.get_file(
    "persons.jpg",
    "https://www.saltwire.com/media/photologue/photos/cache/STJ-A01-28102019-RawlinsCrossPedestrians3_large.jpg")

In [6]:
image = file
img_raw = tf.image.decode_image(open(image, 'rb').read(), channels=3)
img = tf.expand_dims(img_raw, 0)
_,height,width,_ =img.shape
img = tf.image.resize(img, (416, 416))/255

In [7]:
boxes = model(img)

In [8]:
boxes.shape

TensorShape([6, 4])

### Saving the model

In [9]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model_with_NMS/my_model') 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model_with_NMS/my_model/assets


### Test the saved model

In [10]:
new_model = tf.keras.models.load_model('saved_model_with_NMS/my_model')
# Check its architecture
new_model.summary()

Model: "tiny_yol_ov3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BasicBlock1 (BasicBlock)     multiple                  496       
_________________________________________________________________
BasicBlock2 (BasicBlock)     multiple                  4736      
_________________________________________________________________
BasicBlock3 (BasicBlock)     multiple                  18688     
_________________________________________________________________
BasicBlock4 (BasicBlock)     multiple                  74240     
_________________________________________________________________
BasicBlock5 (BasicBlock)     multiple                  295936    
_________________________________________________________________
BasicBlock6 (BasicBlock)     multiple                  1181696   
_________________________________________________________________
BasicBlock7 (BasicBlock)     multiple                 

In [11]:
file = tf.keras.utils.get_file(
    "persons.jpg",
    "https://www.saltwire.com/media/photologue/photos/cache/STJ-A01-28102019-RawlinsCrossPedestrians3_large.jpg")

In [12]:
image = file
img_raw = tf.image.decode_image(open(image, 'rb').read(), channels=3)
img = tf.expand_dims(img_raw, 0)
_,height,width,_ =img.shape
img = tf.image.resize(img, (416, 416))/255

In [14]:
ayuda= model(img)
print(ayuda)

tf.Tensor(
[[0.24873672 0.33036783 0.37678713 0.91031384]
 [0.47777706 0.2421627  0.611129   0.8420085 ]
 [0.6335489  0.28631282 0.7600512  0.7978145 ]
 [0.08708014 0.32900137 0.1485187  0.5255371 ]
 [0.4022988  0.25115263 0.5324107  0.83888245]
 [0.67619    0.15234086 0.8719264  0.78450406]], shape=(6, 4), dtype=float32)


In [15]:
ayuda2= new_model(img)
print(ayuda2)

tf.Tensor(
[[0.24873674 0.33036786 0.37678713 0.9103138 ]
 [0.47777706 0.2421627  0.611129   0.8420085 ]
 [0.6335489  0.28631288 0.7600512  0.7978144 ]
 [0.08708014 0.32900137 0.1485187  0.5255371 ]
 [0.40229878 0.25115263 0.5324107  0.83888245]
 [0.6761901  0.15234086 0.87192637 0.78450406]], shape=(6, 4), dtype=float32)


In [18]:
%timeit model(img)

11.2 ms ± 96.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%timeit new_model(img)

9.75 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
